# COE379L Project 3: Transformer Model Fine-Tuning and Evaluation

## Cross-Model Comparison for News Topic Classification

This notebook covers:
- RoBERTa-base model fine-tuning
- Hugging Face Transformers integration
- Model evaluation and performance metrics
- Training time and inference latency measurement
- Comparison with classical models


## 1. Import Required Libraries


In [ ]:
# Standard library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import warnings
import os
warnings.filterwarnings('ignore')

# PyTorch
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

# Hugging Face
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from datasets import load_dataset

# Scikit-learn for metrics
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    log_loss,
    confusion_matrix,
    classification_report
)

# Set style for visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")

print("\nLibraries imported successfully!")


## 2. Load and Prepare Data


In [ ]:
# Load AG News dataset from Hugging Face
print("Loading AG News dataset...")
dataset = load_dataset("ag_news")

# Extract train and test splits
train_data = dataset['train']
test_data = dataset['test']

print(f"Training samples: {len(train_data):,}")
print(f"Test samples: {len(test_data):,}")

# Class labels
class_labels = ['World', 'Sports', 'Business', 'Sci/Tech']
num_labels = len(class_labels)
print(f"Number of classes: {num_labels}")
print(f"Classes: {class_labels}")

# Show sample
print("\nSample data:")
print(train_data[0])


## 3. Initialize RoBERTa Tokenizer and Model


In [ ]:
# Model name - using RoBERTa-base as specified in requirements
model_name = "roberta-base"

print(f"Loading tokenizer and model: {model_name}")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels
)

# Move model to device
model = model.to(device)

print(f"Model loaded successfully!")
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

# Test tokenization
sample_text = train_data[0]['text']
print(f"\nSample text: {sample_text[:100]}...")
encoded = tokenizer(sample_text, truncation=True, padding='max_length', max_length=512, return_tensors='pt')
print(f"Tokenized shape: {encoded['input_ids'].shape}")


## 4. Preprocess Dataset for Training


In [ ]:
def tokenize_function(examples):
    """Tokenize the text data"""
    return tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=512,  # RoBERTa max length
        return_tensors=None  # Return as lists, not tensors
    )

print("Tokenizing training data...")
train_tokenized = train_data.map(
    tokenize_function,
    batched=True,
    remove_columns=['text']  # Remove original text column
)

print("Tokenizing test data...")
test_tokenized = test_data.map(
    tokenize_function,
    batched=True,
    remove_columns=['text']
)

print(f"\nTokenization complete!")
print(f"Training features: {train_tokenized.column_names}")
print(f"Test features: {test_tokenized.column_names}")

# Set format for PyTorch
train_tokenized.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_tokenized.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

print("Dataset formatted for PyTorch!")


## 5. Define Metrics Function


In [ ]:
def compute_metrics(eval_pred):
    """Compute metrics for evaluation"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    # Calculate metrics
    accuracy = accuracy_score(labels, predictions)
    f1_macro = f1_score(labels, predictions, average='macro')
    
    return {
        'accuracy': accuracy,
        'f1_macro': f1_macro
    }

print("Metrics function defined!")


## 6. Configure Training Arguments


In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./roberta-ag-news',
    num_train_epochs=3,  # Start with 3 epochs, can adjust
    per_device_train_batch_size=16,  # Adjust based on GPU memory
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=500,
    evaluation_strategy="epoch",  # Evaluate at end of each epoch
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    save_total_limit=2,  # Keep only 2 best models
    fp16=torch.cuda.is_available(),  # Use mixed precision if GPU available
    dataloader_num_workers=2 if torch.cuda.is_available() else 0,
    report_to="none",  # Disable wandb/tensorboard
)

print("Training arguments configured!")
print(f"Training epochs: {training_args.num_train_epochs}")
print(f"Batch size (train): {training_args.per_device_train_batch_size}")
print(f"Batch size (eval): {training_args.per_device_eval_batch_size}")
print(f"Mixed precision (FP16): {training_args.fp16}")


## 7. Create Trainer and Fine-Tune Model


In [ ]:
# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,  # Using test set for evaluation during training
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Stop if no improvement for 2 epochs
)

print("Trainer created!")
print("Starting fine-tuning...")
print("=" * 80)

# Record training start time
training_start_time = time.time()

# Train the model
trainer.train()

# Record training end time
training_time = time.time() - training_start_time

print("=" * 80)
print(f"Fine-tuning completed in {training_time:.2f} seconds ({training_time/60:.2f} minutes)")


## 8. Evaluate Model on Test Set


In [ ]:
# Evaluate on test set
print("Evaluating on test set...")
eval_results = trainer.evaluate()

print("\nEvaluation Results:")
print(f"  Accuracy: {eval_results['eval_accuracy']:.4f}")
print(f"  Macro F1-Score: {eval_results['eval_f1_macro']:.4f}")

# Get predictions for detailed metrics
print("\nGenerating predictions...")
predictions = trainer.predict(test_tokenized)

# Extract predictions and labels
y_pred = np.argmax(predictions.predictions, axis=1)
y_test = predictions.label_ids
y_pred_proba = torch.softmax(torch.tensor(predictions.predictions), dim=1).numpy()

# Calculate additional metrics
accuracy = accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')
log_loss_score = log_loss(y_test, y_pred_proba)

print(f"\nDetailed Metrics:")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  Macro F1-Score: {f1_macro:.4f}")
print(f"  Log Loss: {log_loss_score:.4f}")


## 9. Measure Inference Latency


In [ ]:
# Measure inference latency for 1000 samples
print("Measuring inference latency...")

# Sample 1000 test examples
num_samples = 1000
sample_indices = np.random.choice(len(test_tokenized), num_samples, replace=False)
sample_data = test_tokenized.select(sample_indices)

# Warm-up
_ = trainer.predict(sample_data.select(range(10)))

# Measure inference time
inference_start_time = time.time()
_ = trainer.predict(sample_data)
inference_time = time.time() - inference_start_time

# Calculate latency per 1000 samples
inference_latency_per_1k = inference_time

print(f"Inference time for {num_samples} samples: {inference_time:.4f} seconds")
print(f"Inference latency per 1,000 samples: {inference_latency_per_1k:.4f} seconds")


## 10. Save Results


In [ ]:
# Compile results
roberta_results = {
    'Model': 'RoBERTa-base',
    'Accuracy': accuracy,
    'Macro F1-Score': f1_macro,
    'Log Loss': log_loss_score,
    'Training Time (s)': training_time,
    'Inference Latency per 1k (s)': inference_latency_per_1k
}

# Create DataFrame
results_df = pd.DataFrame([roberta_results])

print("=" * 80)
print("ROBERTA MODEL - RESULTS SUMMARY")
print("=" * 80)
print(results_df.to_string(index=False))
print("=" * 80)

# Save results
results_df.to_csv('data/roberta_results.csv', index=False)
print("\nResults saved to data/roberta_results.csv")

# Save model
print("\nSaving model...")
trainer.save_model('data/roberta_ag_news_model')
tokenizer.save_pretrained('data/roberta_ag_news_model')
print("Model saved to data/roberta_ag_news_model/")


## 11. Confusion Matrix


In [ ]:
# Create confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_labels, yticklabels=class_labels,
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix - RoBERTa-base', fontsize=14, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.savefig('data/roberta_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()
print("Confusion matrix saved to data/roberta_confusion_matrix.png")

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=class_labels))


## 12. Compare with Classical Models

Load classical models results and create comparison.


In [ ]:
# Load classical models results if available
classical_results_path = 'data/classical_models_results.csv'

if os.path.exists(classical_results_path):
    classical_df = pd.read_csv(classical_results_path)
    
    # Combine results
    all_results = pd.concat([classical_df, results_df], ignore_index=True)
    
    print("=" * 80)
    print("ALL MODELS - COMPARISON")
    print("=" * 80)
    print(all_results.to_string(index=False))
    print("=" * 80)
    
    # Save combined results
    all_results.to_csv('data/all_models_results.csv', index=False)
    print("\nCombined results saved to data/all_models_results.csv")
    
    # Visualization: F1-Score comparison
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    models = all_results['Model'].values
    f1_scores = all_results['Macro F1-Score'].values
    training_times = all_results['Training Time (s)'].values
    
    # F1-Score comparison
    axes[0].bar(models, f1_scores, color=['steelblue', 'coral', 'lightgreen', 'purple'])
    axes[0].set_title('Macro F1-Score Comparison - All Models', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('Macro F1-Score', fontsize=12)
    axes[0].set_ylim([0, 1])
    axes[0].tick_params(axis='x', rotation=45)
    axes[0].grid(axis='y', alpha=0.3)
    for i, v in enumerate(f1_scores):
        axes[0].text(i, v + 0.01, f'{v:.4f}', ha='center', fontweight='bold')
    
    # Training time comparison
    axes[1].bar(models, training_times, color=['steelblue', 'coral', 'lightgreen', 'purple'])
    axes[1].set_title('Training Time Comparison - All Models', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Training Time (seconds)', fontsize=12)
    axes[1].set_yscale('log')  # Log scale for better visualization
    axes[1].tick_params(axis='x', rotation=45)
    axes[1].grid(axis='y', alpha=0.3)
    for i, v in enumerate(training_times):
        axes[1].text(i, v * 1.2, f'{v:.1f}s', ha='center', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('data/all_models_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("Comparison visualization saved to data/all_models_comparison.png")
    
else:
    print("Classical models results not found. Run 02_Classical_Models.ipynb first.")
    print("RoBERTa results saved separately.")
